In [11]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/ames_raw.csv')

In [3]:
df.head()

,Unnamed: 0,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [4]:
del df['Unnamed: 0']
del df['Order']

In [5]:
df.head()

,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [13]:
# need better naming conventions for these
price_tiers = ['Low', 'Middle', 'High']
quantiles = df['SalePrice'].quantile([.1, .25, .5, .75, .9])
print(quantiles)
#price_values = [df['Sale']]
df['SalePrice'].min()
price_conditions = [
    df['SalePrice'] < df['SalePrice'].quantile(.25),
    (df['SalePrice'] >= df['SalePrice'].quantile(.25)) & (df['SalePrice'] <= df['SalePrice'].quantile(.75)),
    df['SalePrice'] > df['SalePrice'].quantile(.75)
]
df['sale_price_tier'] = np.select(price_conditions, price_tiers)

0.10    105450.0
0.25    129500.0
0.50    160000.0
0.75    213500.0
0.90    281241.7
Name: SalePrice, dtype: float64


In [17]:
df['sale_price_per_sq_ft'] = round(df['SalePrice'] / df['Gr Liv Area'], 2)

In [23]:
seasons = ['Winter', 'Spring', 'Summer', 'Fall']
season_month = [
    df['Mo Sold'].isin([12, 1, 2]),
    df['Mo Sold'].isin([3, 4, 5]),
    df['Mo Sold'].isin([6 , 7, 8]),
    df['Mo Sold'].isin([9, 10, 11])
]
df['season_sold'] = np.select(season_month, seasons)

In [27]:
remodeled_house = (df['Year Remod/Add'] - df['Year Built'] > 2)
df['is_remodeled'] = remodeled_house

In [33]:
recently_remodeled = (df['Yr Sold'] - df['Year Remod/Add'] < 3)
df['recently_remodeled'] = recently_remodeled

In [37]:
df[df['recently_remodeled'] == True].groupby('sale_price_tier').size()

sale_price_tier
High      296
Low        34
Middle    208
dtype: int64